RAG - Loader(UpstageDocumentParseLoader)

In [ ]:
from langchain_upstage import UpstageDocumentParseLoader
from dotenv import load_dotenv
import os

# os.environ["UPSTAGE_API_KEY"] = ""

# .env 파일 로드
load_dotenv()

file_path = "./test_modified.pdf"

loader = UpstageDocumentParseLoader(
    file_path,
    split="page",  # 페이지별로 분할
    output_format="markdown",  # 텍스트 형태로 출력
    ocr="auto",  # 자동 OCR 사용
    coordinates=True,  # 좌표 정보 포함
)

docs = loader.load()

In [ ]:
# print(docs[21].page_content)
# print(docs[0])

Rag - Chunking

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#  문서 자르기 (AI가 읽기 쉬운 크기로 쪼개기)
splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)

# 문서 분할 실행
docs_splitter = splitter.split_documents(docs)

Rag - Embedding

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer
from typing import List


# 기존 임베딩 그대로 사용
hf_embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct",
    model_kwargs={"device": "mps"},
    encode_kwargs={"normalize_embeddings": True},
)

Rag - Vector Store

In [ ]:
# RAG - Vector Store
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    documents=docs_splitter,
    embedding=hf_embeddings,  # 커스텀 임베딩 사용
)

Rag - retriever

In [ ]:
# RAG - retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5},
)

retriever = vectorstore.as_retriever(
    search_type="similarity",  # 유사도 검색
    search_kwargs={"k": 5},  # 상위 5개 결과 반환
)

Rag - Formatting / Prompt

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
# LLM 설정 (스트리밍 활성화)
llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0, streaming=True)


# 간단한 리스트 기반 히스토리 관리
chat_history_list = []


def add_to_history(role, message):
    """히스토리에 메시지 추가"""
    chat_history_list.append(f"{role}: {message}")

    # 최근 6개 메시지만 유지 (3턴)
    if len(chat_history_list) > 6:
        chat_history_list.pop(0)


def get_chat_history(x):
    """현재 히스토리를 문자열로 반환"""
    if not chat_history_list:
        return "이전 대화 없음"
    return "\n".join(chat_history_list)


def clear_history():
    """히스토리 초기화"""
    chat_history_list.clear()


# 문서 포맷팅 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# 일반 함수 정의
def extract_input(data):
    """딕셔너리에서 input 키의 값을 추출하는 함수"""
    return data["input"]


# prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """너는 친절한 한국어 AI 비서야. 
        제공된 문서 내용(context)과 이전 대화 내용을 참고해서 질문에 답해.
        반드시 한국어로만 대답하고, 문서에 없는 내용은 대답하지마.
        
        참고 문서:
        {context}
        
        이전 대화:
        {chat_history}
        """,
        ),
        ("user", "{input}"),
    ]
)

# 체인 구성
rag_multiturn_chain = (
    {
        # 1단계: 사용자 질문으로 관련 문서 검색 후 포맷팅
        "context": extract_input | retriever | format_docs, 
        # 2단계: 원본 질문 전달   
        "input": extract_input, 
        # 3단계: 대화 히스토리 가져오기
        "chat_history": get_chat_history,
    }
    | prompt # 4단계: 프롬프트에 모든 정보 결합
    | llm # 5단계: LLM이 답변 생성
    | StrOutputParser() # 6단계: 문자열로 출력 파싱
)

Rag - Streaming

In [ ]:
def stream_rag_multiturn_response(user_input):
    """RAG + 멀티턴 대화를 스트리밍으로 처리    """

    print(f"🙋 사용자: {user_input}")
    print("🤖 AI: ", end="", flush=True)

    response = ""

    try:
        for chunk in rag_multiturn_chain.stream({"input": user_input}):
            print(chunk, end="", flush=True)
            response += chunk

        print()  # 줄바꿈

        # 히스토리에 대화 추가
        add_to_history("사용자", user_input)
        add_to_history("AI", response)

        return response

    except Exception as e:
        error_msg = f"오류 발생: {str(e)}"
        print(error_msg)
        print(f"상세 오류: {type(e).__name__}: {e}")
        return error_msg

Rag - Chat Interface

In [199]:
# RAG - Streaming
def interactive_chat():
    """대화형 채팅 인터페이스"""
    print("=" * 60)
    print("🤖 RAG 챗봇 시작")
    print("💡 '종료' 입력 시 대화를 끝냅니다.")
    print("=" * 60)

    try:
        while True:
            try:
                user_input = input("\n🙋 사용자: ").strip()

                if not user_input:
                    print("❗ 메시지를 입력해주세요.")
                    continue

                if user_input.lower() == "종료":
                    print("👋 채팅을 종료합니다!")
                    break

                stream_rag_multiturn_response(user_input)

            except KeyboardInterrupt:
                print("\n채팅을 종료합니다!")
                break

    finally:
        clear_history()

Rag - Chat

In [ ]:
# RAG - QnA
if __name__ == "__main__":
    interactive_chat()  # 대화형 모드 실행